<a href="https://colab.research.google.com/github/amr552/Deep-learning-computer-vision/blob/main/plant%20disease%20diagnosis%20pre-rtained%20models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 🔁 Master Script: Run All Pretrained Models on Leaf Dataset

from google.colab import drive
drive.mount('/content/drive')

import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

# 🔗 Dataset Path
train_path = '/content/drive/MyDrive/datasets/PLD_3_Classes_256/Training'
val_path = '/content/drive/MyDrive/datasets/PLD_3_Classes_256/Validation'

IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 10  # You can increase for real experiments

# 📂 Output folders
MODEL_DIR = '/content/drive/MyDrive/Models'
RESULT_DIR = '/content/drive/MyDrive/Results'

# Create output directories if they don't exist
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(RESULT_DIR, exist_ok=True)

# Verify directory existence
if not os.path.exists(MODEL_DIR):
    print(f"Error: Model directory not found at {MODEL_DIR}")
if not os.path.exists(RESULT_DIR):
    print(f"Error: Results directory not found at {RESULT_DIR}")


# 🧪 Define the list of model constructors
model_list = {
    'ResNet50': tf.keras.applications.ResNet50,
    'ResNet101': tf.keras.applications.ResNet101,
    'EfficientNetB0': tf.keras.applications.EfficientNetB0,
    'EfficientNetB3': tf.keras.applications.EfficientNetB3,
    'DenseNet121': tf.keras.applications.DenseNet121,
    'MobileNetV2': tf.keras.applications.MobileNetV2,
    'InceptionV3': tf.keras.applications.InceptionV3,
    'VGG16': tf.keras.applications.VGG16
}

# 🧼 Data generator
datagen = ImageDataGenerator(rescale=1./255)
train_gen = datagen.flow_from_directory(train_path, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='categorical')
val_gen = datagen.flow_from_directory(val_path, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='categorical')
num_classes = train_gen.num_classes

# Define Early Stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)


for name, constructor in model_list.items():
    print(f'\n🔧 Training {name}...\n')

    # 📌 Load base model
    base = constructor(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    base.trainable = False

    x = base.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.3)(x)
    x = Dense(256, activation='relu')(x)
    output = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=base.input, outputs=output)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # 🏋️ Train
    history = model.fit(train_gen, validation_data=val_gen, epochs=EPOCHS, verbose=1, callbacks=[early_stopping])

    # 💾 Save model
    model.save(f'{MODEL_DIR}/model_{name}.h5')

    # 📊 Plot
    plt.figure(figsize=(10,4))
    plt.subplot(1,2,1)
    plt.plot(history.history['accuracy'], label='train')
    plt.plot(history.history['val_accuracy'], label='val')
    plt.title(f'{name} Accuracy')
    plt.legend()
    plt.subplot(1,2,2)
    plt.plot(history.history['loss'], label='train')
    plt.plot(history.history['val_loss'], label='val')
    plt.title(f'{name} Loss')
    plt.legend()
    plt.savefig(f'{RESULT_DIR}/{name}_results.png')
    plt.close()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 3251 images belonging to 4 classes.
Found 416 images belonging to 4 classes.

🔧 Training ResNet50...

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - accuracy: 0.3574 - loss: 1.1696

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


102/102 ━━━━━━━━━━━━━━━━━━━━ 881s 9s/step - accuracy: 0.3576 - loss: 1.1692 - val_accuracy: 0.3630 - val_loss: 1.1683
Epoch 2/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 651s 6s/step - accuracy: 0.4139 - loss: 1.0912 - val_accuracy: 0.4904 - val_loss: 0.9992
Epoch 3/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 598s 6s/step - accuracy: 0.4507 - loss: 1.0191 - val_accuracy: 0.5601 - val_loss: 0.9620
Epoch 4/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 604s 6s/step - accuracy: 0.4798 - loss: 1.0161 - val_accuracy: 0.5192 - val_loss: 0.9673
Epoch 5/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 607s 6s/step - accuracy: 0.5165 - loss: 0.9691 - val_accuracy: 0.4255 - val_loss: 1.0465
Epoch 6/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 627s 6s/step - accuracy: 0.5387 - loss: 0.9602 - val_accuracy: 0.5048 - val_loss: 0.9816


...


🔧 Training EfficientNetB3...

43941136/43941136 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 542s 5s/step - accuracy: 0.3357 - loss: 1.1646 - val_accuracy: 0.3918 - val_loss: 1.0996
Epoch 2/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 555s 5s/step - accuracy: 0.3841 - loss: 1.1080 - val_accuracy: 0.3918 - val_loss: 1.1062
Epoch 3/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 504s 5s/step - accuracy: 0.4028 - loss: 1.0895 - val_accuracy: 0.3918 - val_loss: 1.0970
Epoch 4/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 540s 5s/step - accuracy: 0.4139 - loss: 1.0850 - val_accuracy: 0.3918 - val_loss: 1.0823
Epoch 5/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 559s 5s/step - accuracy: 0.3728 - loss: 1.0952 - val_accuracy: 0.3918 - val_loss: 1.0846
Epoch 6/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 560s 5s/step - accuracy: 0.3858 - loss: 1.0879 - val_accuracy: 0.3918 - val_loss: 1.0851
Epoch 7/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 533s 5s/step - accuracy: 0.4138 - loss: 1.0719 - val_accuracy: 0.4591 - val_loss: 1.0865



🔧 Training DenseNet121...

29084464/29084464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 618s 6s/step - accuracy: 0.7576 - loss: 0.6044 - val_accuracy: 0.9327 - val_loss: 0.1836
Epoch 2/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 618s 6s/step - accuracy: 0.9004 - loss: 0.2597 - val_accuracy: 0.9375 - val_loss: 0.1599
Epoch 3/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 603s 6s/step - accuracy: 0.9180 - loss: 0.2195 - val_accuracy: 0.9519 - val_loss: 0.1353
Epoch 4/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 616s 6s/step - accuracy: 0.9168 - loss: 0.2173 - val_accuracy: 0.9567 - val_loss: 0.1308
Epoch 5/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 620s 6s/step - accuracy: 0.9348 - loss: 0.1751 - val_accuracy: 0.9639 - val_loss: 0.1129
Epoch 6/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 622s 6s/step - accuracy: 0.9317 - loss: 0.1746 - val_accuracy: 0.9591 - val_loss: 0.1195
Epoch 7/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 625s 6s/step - accuracy: 0.9387 - loss: 0.1685 - val_accuracy: 0.9615 - val_loss: 0.1135
Epoch 8/10
102/102


🔧 Training MobileNetV2...

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 167s 2s/step - accuracy: 0.7719 - loss: 0.5553 - val_accuracy: 0.9255 - val_loss: 0.1857
Epoch 2/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 156s 2s/step - accuracy: 0.9262 - loss: 0.2155 - val_accuracy: 0.9279 - val_loss: 0.1916
Epoch 3/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 173s 2s/step - accuracy: 0.9248 - loss: 0.1963 - val_accuracy: 0.9399 - val_loss: 0.1760
Epoch 4/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 159s 2s/step - accuracy: 0.9339 - loss: 0.1770 - val_accuracy: 0.9351 - val_loss: 0.1708
Epoch 5/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 157s 2s/step - accuracy: 0.9525 - loss: 0.1463 - val_accuracy: 0.9303 - val_loss: 0.2105
Epoch 6/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 158s 2s/step - accuracy: 0.9552 - loss: 0.1221 - val_accuracy: 0.9375 - val_loss: 0.1908
Epoch 7/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 164s 2s/step - accuracy: 0.9482 - loss: 0.1344 - val_accuracy: 0.8726 - val_loss: 0.3570



🔧 Training InceptionV3...

87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Epoch 1/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 477s 5s/step - accuracy: 0.6918 - loss: 0.7558 - val_accuracy: 0.8918 - val_loss: 0.2662
Epoch 2/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 454s 4s/step - accuracy: 0.8626 - loss: 0.3406 - val_accuracy: 0.8918 - val_loss: 0.2398
Epoch 3/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 420s 4s/step - accuracy: 0.9002 - loss: 0.2720 - val_accuracy: 0.9014 - val_loss: 0.2224
Epoch 4/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 425s 4s/step - accuracy: 0.9056 - loss: 0.2692 - val_accuracy: 0.9183 - val_loss: 0.2078
Epoch 5/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 454s 4s/step - accuracy: 0.9003 - loss: 0.2503 - val_accuracy: 0.8990 - val_loss: 0.2144
Epoch 6/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 428s 4s/step - accuracy: 0.8994 - loss: 0.2340 - val_accuracy: 0.8798 - val_loss: 0.2670
Epoch 7/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 426s 4s/step - accuracy: 0.9106 - loss: 0.2314 - val_accuracy: 0.8846 - val_loss: 0.2668



🔧 Training VGG16...

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Epoch 1/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 2004s 20s/step - accuracy: 0.5024 - loss: 1.0350 - val_accuracy: 0.7404 - val_loss: 0.6733
Epoch 2/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 2032s 20s/step - accuracy: 0.7134 - loss: 0.6813 - val_accuracy: 0.8317 - val_loss: 0.5297
Epoch 3/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 1990s 20s/step - accuracy: 0.7638 - loss: 0.5740 - val_accuracy: 0.8005 - val_loss: 0.5546
Epoch 4/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 1990s 20s/step - accuracy: 0.7806 - loss: 0.5446 - val_accuracy: 0.8173 - val_loss: 0.5055
Epoch 5/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 1986s 19s/step - accuracy: 0.7893 - loss: 0.5388 - val_accuracy: 0.8606 - val_loss: 0.4215
Epoch 6/10
 32/102 ━━━━━━━━━━━━━━━━━━━━ 20:13 17s/step - accuracy: 0.7733 - loss: 0.5579